In [115]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

In [116]:
#Samedevice Tracking
gsheet_url = "https://docs.google.com/spreadsheets/d/1YJBHgfNr8M95Jsm2pcrfxGg-fDDP0-b8XSo2xMsOs3Y/edit#gid=654908229"

In [71]:
#シート上のデータ消去
for i in range(0,1):
    gs = GSheet()
    gs.clear_sheets(gsheet_url, [i])

INFO:ailab_tools.utils.gsheet:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.utils.gsuite:created OAuth2 credentials.
INFO:googleapiclient.discovery:URL being requested: GET https://www.googleapis.com/discovery/v1/apis/sheets/v4/rest
INFO:googleapiclient.discovery:URL being requested: GET https://sheets.googleapis.com/v4/spreadsheets/1YJBHgfNr8M95Jsm2pcrfxGg-fDDP0-b8XSo2xMsOs3Y?alt=json
INFO:googleapiclient.discovery:URL being requested: POST https://sheets.googleapis.com/v4/spreadsheets/1YJBHgfNr8M95Jsm2pcrfxGg-fDDP0-b8XSo2xMsOs3Y/values/UU%E6%95%B0%21A1%3AAA1001:clear?alt=json


In [72]:
samedevice_table = 'segment.samedevice_user_list_by_drawbridge'
print(samedevice_table)

segment.samedevice_user_list_by_drawbridge


## Samedeviceトラッキング

In [73]:
start_date = "2019-09-13"
end_date = "2019-10-12"

### UU数

Other

In [74]:
#%%slack_notify @yusuke_akada
query = (
    f"""
       select
        "9_Other" as cc
        ,ad_type
        ,ads.is_mobius
        ,count(distinct smn_uid) as imp_uu
        from dsplog.adserver ads
        left join dm.hierarchies hie on ads.target_id = hie.target_id
        where 
        concat_ws('-',ads.year,ads.month,ads.day) between '{start_date}' and '{end_date}'
        and ads.user_agent_os_family in('iOS','Android')
        and action_type in ('I')
        and ads.is_app=true
        and smn_uid not in (select smn_uid from {samedevice_table})
        group by 1,2,3
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        df_all = ir.sql_to_pandas(query)
        gs = GSheet()
        sheet_index = 0
        col_offset=0
        #gs.append_data(gsheet_url, sheet_index, df_all, col_offset)
        gs.add_data(gsheet_url, sheet_index, df_all, [0,0],True)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

Drawbridge_samedevice(先行配信分）

In [77]:
#%%slack_notify @yusuke_akada
query = (
    f"""
    select
    "1_Drawbridge_pre" as cc
    ,ad_type
    ,ads.is_mobius
    ,count(distinct smn_uid) as imp_uu
    from dsplog.adserver ads
    left join dm.hierarchies hie on ads.target_id = hie.target_id
    where 
    concat_ws('-',ads.year,ads.month,ads.day) between '{start_date}' and '{end_date}'
    and ads.user_agent_os_family in('iOS','Android')
    and action_type in ('I')
    and ads.is_app=true
    and smn_uid in (select smn_uid from {samedevice_table} where create_date = '2019-08-10')
    group by 1,2,3
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_drawbridge = ir.sql_to_pandas(query)
    gs = GSheet()
    sheet_index = 0
    col_offset=0
    gs.append_data(gsheet_url, sheet_index, df_drawbridge, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

Drawbridge_samedevice(9月追加分）

In [85]:
#%%slack_notify @yusuke_akada
query = (
    f"""
    select
    "2_Drawbridge" as cc
    ,ad_type
    ,ads.is_mobius
    ,count(distinct smn_uid) as imp_uu
    from dsplog.adserver ads
    left join dm.hierarchies hie on ads.target_id = hie.target_id
    where 
    concat_ws('-',ads.year,ads.month,ads.day) between '{start_date}' and '{end_date}'
    and ads.user_agent_os_family in('iOS','Android')
    and action_type in ('I')
    and ads.is_app=true
    and smn_uid in (select smn_uid from {samedevice_table} where create_date in ('2019-09-14','2019-09-25'))
    group by 1,2,3
    """
)
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_drawbridge = ir.sql_to_pandas(query)
    gs = GSheet()
    sheet_index = 0
    col_offset=0
    gs.append_data(gsheet_url, sheet_index, df_drawbridge, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

### 売上

In [117]:
start_date = "2019-10-27"
end_date = "2019-10-31"

全体

In [118]:
#%%slack_notify @yusuke_akada
for i in range(1,2):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
        f"""
       select
       concat_ws('-', ads.year, ads.month, ads.day) ymd
        ,"9_Other" as cc
        ,ad_type
        ,ads.is_mobius
        ,ads.ssp_id
        ,sum(case when ads.action_type = 'I' then 1 else 0 end) imp
        ,sum(case when ads.action_type = 'C' then 1 else 0 end) click
        ,sum(case when ads.action_type = 'I' then ads.winning_price else 0 end)/1000000 win_price
        ,sum(ads.winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(ads.winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when cv.action_type = 'CTV1' then 1 else 0 end) ctv1
        ,sum(case when cv.action_type = 'CTV2' then 1 else 0 end) ctv2
        ,sum(case when cv.action_type in ('CTV1','CTV2') then 1 else 0 end) ctv
        ,sum(case when cv.action_type = 'VTV1' then 1 else 0 end) vtv1
        ,sum(case when cv.action_type = 'VTV2' then 1 else 0 end) vtv2
        ,sum(case when cv.action_type in ('VTV1','VTV2') then 1 else 0 end) vtv
        from dsplog.adserver ads
        left join dm.hierarchies hie on ads.target_id = hie.target_id
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
        left join (select logicad_request_id,action_type from dsplog.adserver_conversion 
                    where concat_ws('-',year,month,day) between '{start_date}' and '{end_date}') cv
                    on ads.logicad_request_id = cv.logicad_request_id
        where 
        concat_ws('-',ads.year,ads.month,ads.day) between '{start_date}' and '{end_date}'
        and ads.user_agent_os_family in('iOS','Android') 
        and ads.is_app=true
        and ads.smn_uid not in (select smn_uid from {samedevice_table})
        group by 1,2,3,4,5
        order by ymd
        """
        )
        
        df_all = ir.sql_to_pandas(query)
        df_all = df_all.astype({'win_price':float, 'net_sales':float, 'gross_sales':float})
        gs = GSheet()
        sheet_index = 1
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_all, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

Drawbridge_samedevice(先行配信分）

In [113]:
#%%slack_notify @yusuke_akada
for i in range(1,2):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
        f"""
        select
        concat_ws('-', ads.year, ads.month, ads.day) ymd
        ,"1_Drawbridge_pre" as cc
        ,ad_type
        ,ads.is_mobius
        ,ads.ssp_id
        ,sum(case when ads.action_type = 'I' then 1 else 0 end) imp
        ,sum(case when ads.action_type = 'C' then 1 else 0 end) click
        ,sum(case when ads.action_type = 'I' then ads.winning_price else 0 end)/1000000 win_price
        ,sum(ads.winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(ads.winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when cv.action_type = 'CTV1' then 1 else 0 end) ctv1
        ,sum(case when cv.action_type = 'CTV2' then 1 else 0 end) ctv2
        ,sum(case when cv.action_type in ('CTV1','CTV2') then 1 else 0 end) ctv
        ,sum(case when cv.action_type = 'VTV1' then 1 else 0 end) vtv1
        ,sum(case when cv.action_type = 'VTV2' then 1 else 0 end) vtv2
        ,sum(case when cv.action_type in ('VTV1','VTV2') then 1 else 0 end) vtv
        from dsplog.adserver ads
        left join dm.hierarchies hie on ads.target_id = hie.target_id
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
        left join (select logicad_request_id,action_type from dsplog.adserver_conversion 
                    where concat_ws('-',year,month,day) between '{start_date}' and '{end_date}') cv
                    on ads.logicad_request_id = cv.logicad_request_id
        where 
        concat_ws('-',ads.year,ads.month,ads.day) between '{start_date}' and '{end_date}'
        and ads.user_agent_os_family in('iOS','Android') 
        and ads.is_app=true
        and ads.smn_uid in (select smn_uid from {samedevice_table} where create_date = '2019-08-10')
        group by 1,2,3,4,5
        order by ymd
        
        """
        )
        
        df_all = ir.sql_to_pandas(query)
        df_all = df_all.astype({'win_price':float, 'net_sales':float, 'gross_sales':float})
        gs = GSheet()
        sheet_index = 1
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_all, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

Drawbridge_samedevice(9月追加分）

In [119]:
#%%slack_notify @yusuke_akada
for i in range(1,2):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
        f"""
        select
        concat_ws('-', ads.year, ads.month, ads.day) ymd
        ,"2_Drawbridge" as cc
        ,ad_type
        ,ads.is_mobius
        ,ads.ssp_id
        ,sum(case when ads.action_type = 'I' then 1 else 0 end) imp
        ,sum(case when ads.action_type = 'C' then 1 else 0 end) click
        ,sum(case when ads.action_type = 'I' then ads.winning_price else 0 end)/1000000 win_price
        ,sum(ads.winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(ads.winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when cv.action_type = 'CTV1' then 1 else 0 end) ctv1
        ,sum(case when cv.action_type = 'CTV2' then 1 else 0 end) ctv2
        ,sum(case when cv.action_type in ('CTV1','CTV2') then 1 else 0 end) ctv
        ,sum(case when cv.action_type = 'VTV1' then 1 else 0 end) vtv1
        ,sum(case when cv.action_type = 'VTV2' then 1 else 0 end) vtv2
        ,sum(case when cv.action_type in ('VTV1','VTV2') then 1 else 0 end) vtv
        from dsplog.adserver ads
        left join dm.hierarchies hie on ads.target_id = hie.target_id
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
        left join (select logicad_request_id,action_type from dsplog.adserver_conversion 
                    where concat_ws('-',year,month,day) between '{start_date}' and '{end_date}') cv
                    on ads.logicad_request_id = cv.logicad_request_id
        where 
        concat_ws('-',ads.year,ads.month,ads.day) between '{start_date}' and '{end_date}'
        and ads.user_agent_os_family in('iOS','Android') 
        and ads.is_app=true
        and ads.smn_uid in (select smn_uid from {samedevice_table})
        group by 1,2,3,4,5
        order by ymd
        
        """
        )
        
        df_all = ir.sql_to_pandas(query)
        df_all = df_all.astype({'win_price':float, 'net_sales':float, 'gross_sales':float})
        gs = GSheet()
        sheet_index = 1
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_all, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

imp上位案件に絞る

## 効果確認

全体 パターン1　前のテストと同じ案件

#%%slack_notify @yusuke_akada
for i in range(1,2):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
        f"""
       select
        "9_Other" as cc
        ,ad_type
        ,ads.is_mobius
        ,ads.ssp_id
		,ads.advertiser_id
		,hie.advertiser_name
        ,ads.strategy_id
		,hie.strategy_name
        ,sum(case when ads.action_type = 'I' then 1 else 0 end) imp
        ,sum(case when ads.action_type = 'C' then 1 else 0 end) click
        ,sum(case when ads.action_type = 'I' then ads.winning_price else 0 end)/1000000 win_price
        ,sum(ads.winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(ads.winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when cv.action_type = 'CTV1' then 1 else 0 end) ctv1
        ,sum(case when cv.action_type = 'CTV2' then 1 else 0 end) ctv2
        ,sum(case when cv.action_type in ('CTV1','CTV2') then 1 else 0 end) ctv
        ,sum(case when cv.action_type = 'VTV1' then 1 else 0 end) vtv1
        ,sum(case when cv.action_type = 'VTV2' then 1 else 0 end) vtv2
        ,sum(case when cv.action_type in ('VTV1','VTV2') then 1 else 0 end) vtv
        from dsplog.adserver ads
        left join dm.hierarchies hie on ads.target_id = hie.target_id
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
        left join dsplog.adserver_conversion cv on ads.logicad_request_id = cv.logicad_request_id
        where 
        concat_ws('-',ads.year,ads.month,ads.day) between '{start_date}' and '{end_date}'
        and ads.user_agent_os_family in('iOS','Android') 
        and ads.is_app=true
        /*and ads.is_mobius=true*/
        and ads.advertiser_id in (6153,564,1782,5103,9633,6056,9061,476,2663,6986,4272,9397,9660,9592,6416,552,2883,3480,4479,6862)
        and ads.tactics_id in (	61984,	61985,	95071,	232711,	255780,	260173,	260174,	284028,	284029
        ,	299973,	299975,	299976,	299977,	299978,	299979,	299980,	311641,	314069,	327472,	353261,	372426,	372428,	384094,	384098,	384102
        ,	384103,	384109,	384110,	384111,	384112,	384113,	384115,	384116,	384117,	384118,	384119,	384120,	384121,	384717,	387950,	387969
        ,	426595,	426604,	432698,	436652,	436654,	447172,	454519,	455031,	470058,	481115,	481116,	481117,	481131,	483197,	492400,	498750
        ,	498753,	498754,	498755,	510328,	521300,	521301,	523666,	523668,	529653,	529654,	536639,	537497,	545615,	569421,	569423,	583459
        ,	583463,	583470,	583472,	583473,	583474,	583475,	583476,	583858,	583882,	583897,	583900,	583902,	585481,	585482,	585919,	586157
        ,	586158,	589198,	589200,	589205,	598555,	602728,	602729,	602730,	603742,	603744,	604522,	604524,	604526,	604552,	604553,	604583
        ,	605525,	605526,	605528,	606027,	606028,	606039,	606064,	607286,	607289,	607290,	607291,	613437,	613438,	618042,	618049,	618065
        ,	618066,	619049,	625394)
        and ads.smn_uid not in (select smn_uid from {samedevice_table})
        group by 1,2,3,4,5,6,7,8
        having imp > 0 and click > 0
        """
        )
        
        df_all = ir.sql_to_pandas(query)
        df_all = df_all.astype({'win_price':float, 'net_sales':float, 'gross_sales':float})
        gs = GSheet()
        sheet_index = 2
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_all, col_offset)

Drawbridge_samedevice(先行配信分）

#%%slack_notify @yusuke_akada
for i in range(1,2):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
        f"""
       select
        "1_Drabridge_pre" as cc
        ,ad_type
        ,ads.is_mobius
        ,ads.ssp_id
		,ads.advertiser_id
		,hie.advertiser_name
        ,ads.strategy_id
		,hie.strategy_name
        ,sum(case when ads.action_type = 'I' then 1 else 0 end) imp
        ,sum(case when ads.action_type = 'C' then 1 else 0 end) click
        ,sum(case when ads.action_type = 'I' then ads.winning_price else 0 end)/1000000 win_price
        ,sum(ads.winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(ads.winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when cv.action_type = 'CTV1' then 1 else 0 end) ctv1
        ,sum(case when cv.action_type = 'CTV2' then 1 else 0 end) ctv2
        ,sum(case when cv.action_type in ('CTV1','CTV2') then 1 else 0 end) ctv
        ,sum(case when cv.action_type = 'VTV1' then 1 else 0 end) vtv1
        ,sum(case when cv.action_type = 'VTV2' then 1 else 0 end) vtv2
        ,sum(case when cv.action_type in ('VTV1','VTV2') then 1 else 0 end) vtv
        from dsplog.adserver ads
        left join dm.hierarchies hie on ads.target_id = hie.target_id
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
        left join dsplog.adserver_conversion cv on ads.logicad_request_id = cv.logicad_request_id
        where 
        concat_ws('-',ads.year,ads.month,ads.day) between '{start_date}' and '{end_date}'
        and ads.user_agent_os_family in('iOS','Android') 
        and ads.is_app=true
        /*and ads.is_mobius=true*/
        and ads.advertiser_id in (6153,564,1782,5103,9633,6056,9061,476,2663,6986,4272,9397,9660,9592,6416,552,2883,3480,4479,6862)
        and ads.tactics_id in (	61984,	61985,	95071,	232711,	255780,	260173,	260174,	284028,	284029
        ,	299973,	299975,	299976,	299977,	299978,	299979,	299980,	311641,	314069,	327472,	353261,	372426,	372428,	384094,	384098,	384102
        ,	384103,	384109,	384110,	384111,	384112,	384113,	384115,	384116,	384117,	384118,	384119,	384120,	384121,	384717,	387950,	387969
        ,	426595,	426604,	432698,	436652,	436654,	447172,	454519,	455031,	470058,	481115,	481116,	481117,	481131,	483197,	492400,	498750
        ,	498753,	498754,	498755,	510328,	521300,	521301,	523666,	523668,	529653,	529654,	536639,	537497,	545615,	569421,	569423,	583459
        ,	583463,	583470,	583472,	583473,	583474,	583475,	583476,	583858,	583882,	583897,	583900,	583902,	585481,	585482,	585919,	586157
        ,	586158,	589198,	589200,	589205,	598555,	602728,	602729,	602730,	603742,	603744,	604522,	604524,	604526,	604552,	604553,	604583
        ,	605525,	605526,	605528,	606027,	606028,	606039,	606064,	607286,	607289,	607290,	607291,	613437,	613438,	618042,	618049,	618065
        ,	618066,	619049,	625394)
        and ads.smn_uid in (select smn_uid from {samedevice_table} where create_date = '2019-08-10')
        group by 1,2,3,4,5,6,7,8
        having imp > 0 and click > 0
        """
        )
        
        df_all = ir.sql_to_pandas(query)
        df_all = df_all.astype({'win_price':float, 'net_sales':float, 'gross_sales':float})
        gs = GSheet()
        sheet_index = 2
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_all, col_offset)

Drawbridge

#%%slack_notify @yusuke_akada
for i in range(1,2):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
        f"""
        select
        "2_Drawbridge" as cc
        ,ad_type
        ,ads.is_mobius
        ,ads.ssp_id
		,ads.advertiser_id
		,hie.advertiser_name
        ,ads.strategy_id
		,hie.strategy_name
        ,sum(case when ads.action_type = 'I' then 1 else 0 end) imp
        ,sum(case when ads.action_type = 'C' then 1 else 0 end) click
        ,sum(case when ads.action_type = 'I' then ads.winning_price else 0 end)/1000000 win_price
        ,sum(ads.winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(ads.winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when cv.action_type = 'CTV1' then 1 else 0 end) ctv1
        ,sum(case when cv.action_type = 'CTV2' then 1 else 0 end) ctv2
        ,sum(case when cv.action_type in ('CTV1','CTV2') then 1 else 0 end) ctv
        ,sum(case when cv.action_type = 'VTV1' then 1 else 0 end) vtv1
        ,sum(case when cv.action_type = 'VTV2' then 1 else 0 end) vtv2
        ,sum(case when cv.action_type in ('VTV1','VTV2') then 1 else 0 end) vtv
        from dsplog.adserver ads
        left join dm.hierarchies hie on ads.target_id = hie.target_id
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
        left join dsplog.adserver_conversion cv on ads.logicad_request_id = cv.logicad_request_id
        where 
        concat_ws('-',ads.year,ads.month,ads.day) between '{start_date}' and '{end_date}'
        and ads.user_agent_os_family in('iOS','Android') 
        and ads.is_app=true
        /*and ads.is_mobius=true*/
        and ads.advertiser_id in (6153,564,1782,5103,9633,6056,9061,476,2663,6986,4272,9397,9660,9592,6416,552,2883,3480,4479,6862)
        and ads.tactics_id in (	61984,	61985,	95071,	232711,	255780,	260173,	260174,	284028,	284029
        ,	299973,	299975,	299976,	299977,	299978,	299979,	299980,	311641,	314069,	327472,	353261,	372426,	372428,	384094,	384098,	384102
        ,	384103,	384109,	384110,	384111,	384112,	384113,	384115,	384116,	384117,	384118,	384119,	384120,	384121,	384717,	387950,	387969
        ,	426595,	426604,	432698,	436652,	436654,	447172,	454519,	455031,	470058,	481115,	481116,	481117,	481131,	483197,	492400,	498750
        ,	498753,	498754,	498755,	510328,	521300,	521301,	523666,	523668,	529653,	529654,	536639,	537497,	545615,	569421,	569423,	583459
        ,	583463,	583470,	583472,	583473,	583474,	583475,	583476,	583858,	583882,	583897,	583900,	583902,	585481,	585482,	585919,	586157
        ,	586158,	589198,	589200,	589205,	598555,	602728,	602729,	602730,	603742,	603744,	604522,	604524,	604526,	604552,	604553,	604583
        ,	605525,	605526,	605528,	606027,	606028,	606039,	606064,	607286,	607289,	607290,	607291,	613437,	613438,	618042,	618049,	618065
        ,	618066,	619049,	625394)
        and ads.smn_uid in (select smn_uid from {samedevice_table})
        group by 1,2,3,4,5,6,7,8
        having imp > 0 and click > 0
        
        """
        )
        
        df_all = ir.sql_to_pandas(query)
        df_all = df_all.astype({'win_price':float, 'net_sales':float, 'gross_sales':float})
        gs = GSheet()
        sheet_index = 2
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_all, col_offset)

全体 パターン2 ※上位広告主のキャンペーン限定

In [32]:
start_date = "2019-10-01"
end_date = "2019-10-09"

In [33]:
#%%slack_notify @yusuke_akada
for i in range(1,2):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
        f"""
       select
        "9_Other" as cc
        ,ad_type
        ,ads.is_mobius
        ,ads.ssp_id
		,ads.advertiser_id
		,hie.advertiser_name
        ,ads.strategy_id
		,hie.strategy_name
        ,sum(case when ads.action_type = 'I' then 1 else 0 end) imp
        ,sum(case when ads.action_type = 'C' then 1 else 0 end) click
        ,sum(case when ads.action_type = 'I' then ads.winning_price else 0 end)/1000000 win_price
        ,sum(ads.winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(ads.winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when cv.action_type = 'CTV1' then 1 else 0 end) ctv1
        ,sum(case when cv.action_type = 'CTV2' then 1 else 0 end) ctv2
        ,sum(case when cv.action_type in ('CTV1','CTV2') then 1 else 0 end) ctv
        ,sum(case when cv.action_type = 'VTV1' then 1 else 0 end) vtv1
        ,sum(case when cv.action_type = 'VTV2' then 1 else 0 end) vtv2
        ,sum(case when cv.action_type in ('VTV1','VTV2') then 1 else 0 end) vtv
        from dsplog.adserver ads
        left join dm.hierarchies hie on ads.target_id = hie.target_id
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
        left join dsplog.adserver_conversion cv on ads.logicad_request_id = cv.logicad_request_id
        where 
        concat_ws('-',ads.year,ads.month,ads.day) between '{start_date}' and '{end_date}'
        and ads.user_agent_os_family in('iOS','Android') 
        and ads.is_app=true
        /*and ads.is_mobius=true*/
        and ads.smn_uid not in (select smn_uid from {samedevice_table})
        and ad_type in (0,4)
        and ads.advertiser_id in (476,564,1782,2294,2883,3354,5103,5877,6056,6153,6330,6618,6885,7228,7596,8928,9061,9372,9397,9592)
        and ads.strategy_id in (15370,18074,28751,30341,31097,32236,34998,36262,36298,36301,38940,38941,41748
        ,41838,42865,45203,45501,45568,45575,45577,46440,46527,48301,48792,48932,48934,48937,49134,50083,50148,50149,51151,51414
        ,51480,51554,51651,51652,52693,52717,52840,52845,52846,52860,52862,53113,53118,53337,53389,53393)
        group by 1,2,3,4,5,6,7,8
        """
        )
        
        df_all = ir.sql_to_pandas(query)
        df_all = df_all.astype({'win_price':float, 'net_sales':float, 'gross_sales':float})
        gs = GSheet()
        sheet_index = 2
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_all, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

Drawbridge_samedevice(先行配信分）

In [ ]:
#%%slack_notify @yusuke_akada
for i in range(1,2):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
        f"""
       select
        "1_Drabridge_pre" as cc
        ,ad_type
        ,ads.is_mobius
        ,ads.ssp_id
		,ads.advertiser_id
		,hie.advertiser_name
        ,ads.strategy_id
		,hie.strategy_name
        ,sum(case when ads.action_type = 'I' then 1 else 0 end) imp
        ,sum(case when ads.action_type = 'C' then 1 else 0 end) click
        ,sum(case when ads.action_type = 'I' then ads.winning_price else 0 end)/1000000 win_price
        ,sum(ads.winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(ads.winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when cv.action_type = 'CTV1' then 1 else 0 end) ctv1
        ,sum(case when cv.action_type = 'CTV2' then 1 else 0 end) ctv2
        ,sum(case when cv.action_type in ('CTV1','CTV2') then 1 else 0 end) ctv
        ,sum(case when cv.action_type = 'VTV1' then 1 else 0 end) vtv1
        ,sum(case when cv.action_type = 'VTV2' then 1 else 0 end) vtv2
        ,sum(case when cv.action_type in ('VTV1','VTV2') then 1 else 0 end) vtv
        from dsplog.adserver ads
        left join dm.hierarchies hie on ads.target_id = hie.target_id
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
        left join dsplog.adserver_conversion cv on ads.logicad_request_id = cv.logicad_request_id
        where 
        concat_ws('-',ads.year,ads.month,ads.day) between '{start_date}' and '{end_date}'
        and ads.user_agent_os_family in('iOS','Android') 
        and ads.is_app=true
        /*and ads.is_mobius=true*/
        and ad_type in (0,4)
        and ads.advertiser_id in (10033,6153,7228,6862,5103,6986,2663,9372,9301,8889,7528,4272,7912,2294,8049,101,6330,9507,9620,1128)
        and ads.strategy_id in (15370,18074,28751,30341,31097,32236,34998,36262,36298,36301,38940,38941,41748
        ,41838,42865,45203,45501,45568,45575,45577,46440,46527,48301,48792,48932,48934,48937,49134,50083,50148,50149,51151,51414
        ,51480,51554,51651,51652,52693,52717,52840,52845,52846,52860,52862,53113,53118,53337,53389,53393
        and ads.smn_uid in (select smn_uid from {samedevice_table} where create_date = '2019-08-10')
        group by 1,2,3,4,5,6,7,8
        """
        )
        
        df_all = ir.sql_to_pandas(query)
        df_all = df_all.astype({'win_price':float, 'net_sales':float, 'gross_sales':float})
        gs = GSheet()
        sheet_index = 2
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_all, col_offset)

Drawbridge

In [ ]:
#%%slack_notify @yusuke_akada
for i in range(1,2):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
        f"""
        select
        "2_Drawbridge" as cc
        ,ad_type
        ,ads.is_mobius
        ,ads.ssp_id
		,ads.advertiser_id
		,hie.advertiser_name
        ,ads.strategy_id
		,hie.strategy_name
        ,sum(case when ads.action_type = 'I' then 1 else 0 end) imp
        ,sum(case when ads.action_type = 'C' then 1 else 0 end) click
        ,sum(case when ads.action_type = 'I' then ads.winning_price else 0 end)/1000000 win_price
        ,sum(ads.winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(ads.winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when cv.action_type = 'CTV1' then 1 else 0 end) ctv1
        ,sum(case when cv.action_type = 'CTV2' then 1 else 0 end) ctv2
        ,sum(case when cv.action_type in ('CTV1','CTV2') then 1 else 0 end) ctv
        ,sum(case when cv.action_type = 'VTV1' then 1 else 0 end) vtv1
        ,sum(case when cv.action_type = 'VTV2' then 1 else 0 end) vtv2
        ,sum(case when cv.action_type in ('VTV1','VTV2') then 1 else 0 end) vtv
        from dsplog.adserver ads
        left join dm.hierarchies hie on ads.target_id = hie.target_id
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
        left join dsplog.adserver_conversion cv on ads.logicad_request_id = cv.logicad_request_id
        where 
        concat_ws('-',ads.year,ads.month,ads.day) between '{start_date}' and '{end_date}'
        and ads.user_agent_os_family in('iOS','Android') 
        and ads.is_app=true
        /*and ads.is_mobius=true*/
        and ad_type in (0,4)
        and ads.advertiser_id in (476,564,1782,2294,2883,3354,5103,5877,6056,6153,6330,6618,6885,7228,7596,8928,9061,9372,9397,9592)
        and ads.strategy_id in (15370,18074,28751,30341,31097,32236,34998,36262,36298,36301,38940,38941,41748
        ,41838,42865,45203,45501,45568,45575,45577,46440,46527,48301,48792,48932,48934,48937,49134,50083,50148,50149,51151,51414
        ,51480,51554,51651,51652,52693,52717,52840,52845,52846,52860,52862,53113,53118,53337,53389,53393)
        and ads.smn_uid in (select smn_uid from {samedevice_table})
        group by 1,2,3,4,5,6,7,8
        
        """
        )
        
        df_all = ir.sql_to_pandas(query)
        df_all = df_all.astype({'win_price':float, 'net_sales':float, 'gross_sales':float})
        gs = GSheet()
        sheet_index = 2
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_all, col_offset)

全体 パターン3 ※上位広告主の配信設定単位で抽出　集計はキャンペーン

In [210]:
start_date = "2019-09-13"
end_date = "2019-09-30"

In [211]:
#%%slack_notify @yusuke_akada
for i in range(1,2):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
        f"""
       select
        "9_Other" as cc
        ,ad_type
        ,ads.is_mobius
        ,ads.ssp_id
		,ads.advertiser_id
		,hie.advertiser_name
        ,ads.strategy_id
		,hie.strategy_name
        ,sum(case when ads.action_type = 'I' then 1 else 0 end) imp
        ,sum(case when ads.action_type = 'C' then 1 else 0 end) click
        ,sum(case when ads.action_type = 'I' then ads.winning_price else 0 end)/1000000 win_price
        ,sum(ads.winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(ads.winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when cv.action_type = 'CTV1' then 1 else 0 end) ctv1
        ,sum(case when cv.action_type = 'CTV2' then 1 else 0 end) ctv2
        ,sum(case when cv.action_type in ('CTV1','CTV2') then 1 else 0 end) ctv
        ,sum(case when cv.action_type = 'VTV1' then 1 else 0 end) vtv1
        ,sum(case when cv.action_type = 'VTV2' then 1 else 0 end) vtv2
        ,sum(case when cv.action_type in ('VTV1','VTV2') then 1 else 0 end) vtv
        from dsplog.adserver ads
        left join dm.hierarchies hie on ads.target_id = hie.target_id
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
        left join dsplog.adserver_conversion cv on ads.logicad_request_id = cv.logicad_request_id
        where 
        concat_ws('-',ads.year,ads.month,ads.day) between '{start_date}' and '{end_date}'
        and ads.user_agent_os_family in('iOS','Android') 
        and ads.is_app=true
        /*and ads.is_mobius=true*/
        and ads.smn_uid not in (select smn_uid from {samedevice_table})
        and ad_type in (0,4)
        and ads.advertiser_id in (476,564,1782,2294,2883,3354,5103,5877,6056,6153,6330,6618,6885,7228,7596,8928,9061,9372,9397,9592)
        and ads.tactics_id in (
         90086,90090,91955,91956,95071,111108,159978,197170,202284,202516,210995,210997,210998,255780,255783,260172,260173,260174,299975,299977,311642,319997
        ,319998,319999,320000,320001,320002,320006,327472,360856,381611,384102,384103,384105,384106,384109,384113,384116,384117,384118,384119,384120,384121
        ,425741,426595,426600,447994,468101,468102,469644,469645,469646,469647,481117,481131,482153,482226,482229,483196,483197,494000,496921,496922,496923
        ,501468,506541,506542,507870,507871,507910,522669,522670,522671,522672,522699,522700,522701,529653,529654,530019,530149,530150,536633,536634,536636
        ,536637,536638,536639,537357,537367,537406,540987,540989,543039,543040,544180,544184,549642,550509,563915,573617
        ,573618,573622,573623,583469,583470,583471,583472,583473,583474,583475,583476,583895,583896,583897,583899,583900
        ,583902,585481,585482,585658,585659,585918,585919,589076,589078,589079,589080,589197,589198,589213,594030,603741
        ,603742,603743,603744,603745,603746,604524,604552,604553,604554,604555,606064,619032,619033,622814,622816,622819
        ,623603,623604,623606,623677,626173,626174,626175,626192,626193,626214,626229,626230,629805,633610,633632,633633
        ,633655,633680,641684,642277,644015,644025,644026,644027,644028,644029,644030,644090,644102,644104,647891,647939
        ,650450,650455,650458,651430,651434,651435,651436,651437,651464,651465)
        group by 1,2,3,4,5,6,7,8
        having imp > 0 and click > 0
        """
        )
        
        df_all = ir.sql_to_pandas(query)
        df_all = df_all.astype({'win_price':float, 'net_sales':float, 'gross_sales':float})
        gs = GSheet()
        sheet_index = 4
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_all, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

Drawbridge_samedevice(先行配信分）

In [212]:
#%%slack_notify @yusuke_akada
for i in range(1,2):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
        f"""
       select
        "1_Drabridge_pre" as cc
        ,ad_type
        ,ads.is_mobius
        ,ads.ssp_id
		,ads.advertiser_id
		,hie.advertiser_name
        ,ads.strategy_id
		,hie.strategy_name
        ,sum(case when ads.action_type = 'I' then 1 else 0 end) imp
        ,sum(case when ads.action_type = 'C' then 1 else 0 end) click
        ,sum(case when ads.action_type = 'I' then ads.winning_price else 0 end)/1000000 win_price
        ,sum(ads.winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(ads.winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when cv.action_type = 'CTV1' then 1 else 0 end) ctv1
        ,sum(case when cv.action_type = 'CTV2' then 1 else 0 end) ctv2
        ,sum(case when cv.action_type in ('CTV1','CTV2') then 1 else 0 end) ctv
        ,sum(case when cv.action_type = 'VTV1' then 1 else 0 end) vtv1
        ,sum(case when cv.action_type = 'VTV2' then 1 else 0 end) vtv2
        ,sum(case when cv.action_type in ('VTV1','VTV2') then 1 else 0 end) vtv
        from dsplog.adserver ads
        left join dm.hierarchies hie on ads.target_id = hie.target_id
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
        left join dsplog.adserver_conversion cv on ads.logicad_request_id = cv.logicad_request_id
        where 
        concat_ws('-',ads.year,ads.month,ads.day) between '{start_date}' and '{end_date}'
        and ads.user_agent_os_family in('iOS','Android') 
        and ads.is_app=true
        /*and ads.is_mobius=true*/
        and ad_type in (0,4)
        and ads.advertiser_id in (476,564,1782,2294,2883,3354,5103,5877,6056,6153,6330,6618,6885,7228,7596,8928,9061,9372,9397,9592)
        and ads.tactics_id in (
         90086,90090,91955,91956,95071,111108,159978,197170,202284,202516,210995,210997,210998,255780,255783,260172,260173,260174,299975,299977,311642,319997
        ,319998,319999,320000,320001,320002,320006,327472,360856,381611,384102,384103,384105,384106,384109,384113,384116,384117,384118,384119,384120,384121
        ,425741,426595,426600,447994,468101,468102,469644,469645,469646,469647,481117,481131,482153,482226,482229,483196,483197,494000,496921,496922,496923
        ,501468,506541,506542,507870,507871,507910,522669,522670,522671,522672,522699,522700,522701,529653,529654,530019,530149,530150,536633,536634,536636
        ,536637,536638,536639,537357,537367,537406,540987,540989,543039,543040,544180,544184,549642,550509,563915,573617
        ,573618,573622,573623,583469,583470,583471,583472,583473,583474,583475,583476,583895,583896,583897,583899,583900
        ,583902,585481,585482,585658,585659,585918,585919,589076,589078,589079,589080,589197,589198,589213,594030,603741
        ,603742,603743,603744,603745,603746,604524,604552,604553,604554,604555,606064,619032,619033,622814,622816,622819
        ,623603,623604,623606,623677,626173,626174,626175,626192,626193,626214,626229,626230,629805,633610,633632,633633
        ,633655,633680,641684,642277,644015,644025,644026,644027,644028,644029,644030,644090,644102,644104,647891,647939
        ,650450,650455,650458,651430,651434,651435,651436,651437,651464,651465)
        and ads.smn_uid in (select smn_uid from {samedevice_table} where create_date = '2019-08-10')
        group by 1,2,3,4,5,6,7,8
        having imp > 0 and click > 0
        """
        )
        
        df_all = ir.sql_to_pandas(query)
        df_all = df_all.astype({'win_price':float, 'net_sales':float, 'gross_sales':float})
        gs = GSheet()
        sheet_index = 4
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_all, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     

Drawbridge

In [216]:
#%%slack_notify @yusuke_akada
for i in range(1,2):
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        query = (
        f"""
        select
        "2_Drawbridge" as cc
        ,ad_type
        ,ads.is_mobius
        ,ads.ssp_id
		,ads.advertiser_id
		,hie.advertiser_name
        ,ads.strategy_id
		,hie.strategy_name
        ,sum(case when ads.action_type = 'I' then 1 else 0 end) imp
        ,sum(case when ads.action_type = 'C' then 1 else 0 end) click
        ,sum(case when ads.action_type = 'I' then ads.winning_price else 0 end)/1000000 win_price
        ,sum(ads.winning_price*(1+csm.margin/100))/1000000 net_sales
        ,sum(ads.winning_price*(1+csm.margin/100)/(1-cam.margin/100))/1000000 gross_sales
        ,sum(case when cv.action_type = 'CTV1' then 1 else 0 end) ctv1
        ,sum(case when cv.action_type = 'CTV2' then 1 else 0 end) ctv2
        ,sum(case when cv.action_type in ('CTV1','CTV2') then 1 else 0 end) ctv
        ,sum(case when cv.action_type = 'VTV1' then 1 else 0 end) vtv1
        ,sum(case when cv.action_type = 'VTV2' then 1 else 0 end) vtv2
        ,sum(case when cv.action_type in ('VTV1','VTV2') then 1 else 0 end) vtv
        from dsplog.adserver ads
        left join dm.hierarchies hie on ads.target_id = hie.target_id
        left join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
        left join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date 
        left join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
        left join dsplog.adserver_conversion cv on ads.logicad_request_id = cv.logicad_request_id
        where 
        concat_ws('-',ads.year,ads.month,ads.day) between '{start_date}' and '{end_date}'
        and ads.user_agent_os_family in('iOS','Android') 
        and ads.is_app=true
        /*and ads.is_mobius=true*/
        and ad_type in (0,4)
        and ads.advertiser_id in (476,564,1782,2294,2883,3354,5103,5877,6056,6153,6330,6618,6885,7228,7596,8928,9061,9372,9397,9592)
        and ads.tactics_id in (
         90086,90090,91955,91956,95071,111108,159978,197170,202284,202516,210995,210997,210998,255780,255783,260172,260173,260174,299975,299977,311642,319997
        ,319998,319999,320000,320001,320002,320006,327472,360856,381611,384102,384103,384105,384106,384109,384113,384116,384117,384118,384119,384120,384121
        ,425741,426595,426600,447994,468101,468102,469644,469645,469646,469647,481117,481131,482153,482226,482229,483196,483197,494000,496921,496922,496923
        ,501468,506541,506542,507870,507871,507910,522669,522670,522671,522672,522699,522700,522701,529653,529654,530019,530149,530150,536633,536634,536636
        ,536637,536638,536639,537357,537367,537406,540987,540989,543039,543040,544180,544184,549642,550509,563915,573617
        ,573618,573622,573623,583469,583470,583471,583472,583473,583474,583475,583476,583895,583896,583897,583899,583900
        ,583902,585481,585482,585658,585659,585918,585919,589076,589078,589079,589080,589197,589198,589213,594030,603741
        ,603742,603743,603744,603745,603746,604524,604552,604553,604554,604555,606064,619032,619033,622814,622816,622819
        ,623603,623604,623606,623677,626173,626174,626175,626192,626193,626214,626229,626230,629805,633610,633632,633633
        ,633655,633680,641684,642277,644015,644025,644026,644027,644028,644029,644030,644090,644102,644104,647891,647939
        ,650450,650455,650458,651430,651434,651435,651436,651437,651464,651465)
        and ads.smn_uid in (select smn_uid from {samedevice_table})
        group by 1,2,3,4,5,6,7,8
        having imp > 0 and click > 0
        
        """
        )
        
        df_all = ir.sql_to_pandas(query)
        df_all = df_all.astype({'win_price':float, 'net_sales':float, 'gross_sales':float})
        gs = GSheet()
        sheet_index = 4
        col_offset=0
        gs.append_data(gsheet_url, sheet_index, df_all, col_offset)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
     